## 1. **ABTE**
#### 1.1 Запускаем на тестовых данных

Комментарии см. в readme репозитория

In [ ]:
!pip install adapters --q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 3.7 MB/s eta 0:00:00


In [ ]:
from utils_for_abte import clean_data, clean_idx, spans_to_tokens, predictor
from abte import ABTEDataset, ABTEBert, ABTEModel
from transformers import BertModel, BertTokenizer
import torch

tokenizer_config.json:   0%|          | 0.00/401 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.74M [00:00<?, ?B/s]

In [ ]:
!wget -O fine_model.pkl "drive.google.com/u/3/uc?id=1-LjUKtk4ejuFFgDg6PBH0ung0o25XWhR&export=download&confirm=yes" --quiet

In [ ]:
!wget -O adapter_model.pkl "drive.google.com/u/3/uc?id=1-upUgxGn__M-WZHAdOIsbyN0Lf-ULrCy&export=download&confirm=yes" --quiet

In [ ]:
text_path = 'dev_reviews.txt'
asp_path = 'dev_aspects.txt'
# форматируем данные
dev_ids = clean_idx(text_path)
dev_raw = clean_data(text_path, asp_path)

#инициализируем модель
tokenizer_path = 'cointegrated/rubert-tiny2'
tokenizer = BertTokenizer.from_pretrained(tokenizer_path)
fine_model = ABTEModel(tokenizer, False)
adapter_model = ABTEModel(tokenizer, True)

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# сразу записывает в файл в нужном формате
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
predictor(dev_raw, 'fine_model.pkl', fine_model, dev_ids, 'fine-tuning_preds.txt')

In [ ]:
predictor(dev_raw, 'adapter_model.pkl', adapter_model, dev_ids, 'adapter_preds.txt')

#### 1.2 Оцениваем предсказания

код из `Evaluation.ipynb` обернём

In [ ]:
from collections import defaultdict

def evaluator(gold_test_path, pred_test_path):
    gold_aspect_cats = {}
    with open(gold_test_path) as fg:
        for line in fg:
            line = line.rstrip('\r\n').split('\t')
            if line[0] not in gold_aspect_cats:
                gold_aspect_cats[line[0]] = {"starts":[], "ends":[], "cats":[], "sents":[]}
                gold_aspect_cats[line[0]]["starts"].append(int(line[3]))
                gold_aspect_cats[line[0]]["ends"].append(int(line[4]))
                gold_aspect_cats[line[0]]["cats"].append(line[1])
                gold_aspect_cats[line[0]]["sents"].append(line[5])
    full_match, partial_match, full_cat_match, partial_cat_match = 0, 0, 0, 0
    total = 0
    fully_matched_pairs = []
    partially_matched_pairs = []
    with open(pred_test_path) as fp:
        for line in fp:
            total += 1
            line = line.rstrip('\r\n').split('\t')
            start, end = int(line[3]), int(line[4])
            category = line[1]
            doc_gold_aspect_cats = gold_aspect_cats[line[0]]
            if start in doc_gold_aspect_cats["starts"]:
               i = doc_gold_aspect_cats["starts"].index(start)
               if doc_gold_aspect_cats["ends"][i] == end:
                    full_match += 1
                    if doc_gold_aspect_cats["cats"][i] == category:
                        full_cat_match += 1
                    else:
                        partial_cat_match += 1
                    fully_matched_pairs.append(
                       (
                           [
                            doc_gold_aspect_cats["starts"][i],
                            doc_gold_aspect_cats["ends"][i],
                            doc_gold_aspect_cats["cats"][i],
                            doc_gold_aspect_cats["sents"][i]
                            ],
                           line
                       )
                    )
                    continue
            for s_pos in doc_gold_aspect_cats["starts"]:
                if start <= s_pos:
                    i = doc_gold_aspect_cats["starts"].index(s_pos)
                    if doc_gold_aspect_cats["ends"][i] == end:
                       partial_match += 1
                        partially_matched_pairs.append(
                           (
                                [
                                    doc_gold_aspect_cats["starts"][i],
                                    doc_gold_aspect_cats["ends"][i],
                                    doc_gold_aspect_cats["cats"][i],
                                    doc_gold_aspect_cats["sents"][i]
                               ],
                               line
                            )
                        )
                        if doc_gold_aspect_cats["cats"][i] == category:
                            partial_cat_match += 1
                        continue
                    matched = False
                    for e_pos in doc_gold_aspect_cats["ends"][i:]:
                        if s_pos <= end <= e_pos:
                            partial_match += 1
                            partially_matched_pairs.append(
                                (
                                    [
                                    doc_gold_aspect_cats["starts"][i],
                                    doc_gold_aspect_cats["ends"][i],
                                    doc_gold_aspect_cats["cats"][i],
                                    doc_gold_aspect_cats["sents"][i]
                                    ],
                                    line
                                )
                            )
                            if doc_gold_aspect_cats["cats"][i] == category:
                                partial_cat_match += 1
                            matched = True
                            break
                    if matched:
                        break
                if start > s_pos:
                    i = doc_gold_aspect_cats["starts"].index(s_pos)
                    if start < doc_gold_aspect_cats["ends"][i] <= end:
                        partial_match += 1
                        partially_matched_pairs.append(
                            (
                                [
                                    doc_gold_aspect_cats["starts"][i],
                                    doc_gold_aspect_cats["ends"][i],
                                    doc_gold_aspect_cats["cats"][i],
                                    doc_gold_aspect_cats["sents"][i]
                                ],
                                line
                            )
                        )
                        if doc_gold_aspect_cats["cats"][i] == category:
                            partial_cat_match += 1
                        break
    gold_size = sum([len(gold_aspect_cats[x]["cats"]) for x in gold_aspect_cats])
    print(f"""
    Full match precision: {full_match / total}
    Full match recall: {full_match / gold_size}
    Partial match ratio in pred: {(full_match + partial_match)  / total}
    Full category accuracy: {full_cat_match / total}
    Partial category accuracy: {(full_cat_match + partial_cat_match) / total}
    """)

# 2. **ABSA**

Двумя способами:
1. out-of-the-box решение с помощью [PyABSA](https://pyabsa.readthedocs.io/en/latest/#).<br>**Проблемы**: есть только три класса тональности (нет *both*), ~~нельзя~~ очень сложно дообучить (свой формат датасетов...)
2. при помощи BERT и маскирования аспектных слов<br>**Проблемы**: долго, лосс начинает стагнироваться, нельзя большие батчи.

## 2.1 *PyABSA*
**NB!** Работает только в Kaggle.

In [ ]:
# !pip install transformers==4.29.0 -q
# !pip install pyabsa==1.16.27 -q
# !conda install -y gdown

In [ ]:
from absa_bert import PyABSAModel

In [ ]:
df_test = pd.read_csv('dev_inverse.tsv', delimiter='\t')
df_train = pd.read_csv('train_inverse.tsv', delimiter='\t')
df_finetuning = pd.read_csv('fine-tuning_preds.txt',
                            delimiter='\t',
                            names=['review_id', 'category', 'span', 'span_start', 'span_end', 'sentiment', 'text']
                            )
df_adapter = pd.read_csv('adapter_preds.txt',
                         delimiter='\t',
                         names=['review_id', 'category', 'span', 'span_start', 'span_end', 'sentiment', 'text']
                         )

In [ ]:
pyabsa_model = PyABSAModel()

In [ ]:
preds_dev, accuracy_dev = pyabsa_model.predict(df_test)
print('Accuracy Dev:', accuracy_dev)

Accuracy Dev: 0.6218487394957983


In [ ]:
preds_train, accuracy_train = pyabsa_model.predict(df_train)
print('Accuracy Train:', accuracy_train)

Accuracy Train: 0.6065504933865211


In [ ]:
pyabsa_model.apply_to_res(df_finetuning).to_csv('fine-tuning_preds_absa.txt', sep='\t')
pyabsa_model.apply_to_res(df_adapter).to_csv('adapter_preds_absa.txt', sep='\t')

## 2.2. *BERT*

In [ ]:
# !pip install transformers -q

Вот ссылки на обученные модельки:

In [ ]:
!wget -O model_epochs3_batch5.pkl 'drive.google.com/u/3/uc?id=1Zsadd-x4ZiODUSj9kIORDQdTMDTgpBcj&export=download&confirm=yes' --quiet
!wget -O model_epochs15_batch5.pkl 'drive.google.com/u/3/uc?id=1m62fpVcmZSdiBpJzpiLzVfam918SncMq&export=download&confirm=yes' --quiet

In [ ]:
from absa_bert import ABSADataset, ABSABert, ABSAModel

In [ ]:
df_dev = pd.read_csv('dev_join.tsv', delimiter='\t', index_col=0)
df_train = pd.read_csv('train_join.tsv', delimiter='\t', index_col=0)

Так выглядит тренировка:

In [ ]:
n_epoch = 15
batch_size = 5
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
modelABSA = ABSAModel()

In [ ]:
modelABSA.train(df_train, n_epoch=n_epoch, batch_size=batch_size, device=device)

Можно взять готовую модельку:

In [ ]:
preds_15, acc_15 = modelABSA.predict(df_dev, 'model_epochs15_batch5.pkl')

In [ ]:
print('Accuracy (epoch 15):', acc_15)

Accuracy (epoch 15): 0.6680672268907563


# 3. **Cats**

In [ ]:
from transformers import AutoModel, AutoTokenizer

model_name = 'numblilbug/food-rubert-sentiment-model'

food_model = AutoModel.from_pretrained(model_name)
food_tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/711M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.57M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]